# **Introduction to Task Guardrails in CrewAI**

This notebook will demonstrate using CrewAI Task Guardrails — a powerful way to add custom validation logic to your agent workflows.

Just like bumpers in bowling, guardrails help your AI agents stay on track by validating the output of a task and deciding whether it needs to retry.

We’ll show you how to:
*   Write a custom guardrail function
*   Attach it to a task
*   Automatically retry until the output meets your criteria

Let’s get started! 🚀

## Setup: Imports and API Key
Before we define our agent and guardrail logic, we import the required modules from CrewAI and initialize the LLM.

We'll also use a search tool (**SerperDevTool**) and the **TaskOutput** object to handle validation logic.

In [1]:
%uv pip install -U -q crewai crewai-tools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from crewai import Agent, Task, LLM, Crew
from typing import Tuple, Union, Dict, Any
from crewai import TaskOutput
from datetime import date
from crewai_tools import SerperDevTool

/Users/tonykipkemboi/Workspace/crewAI-quickstarts/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [3]:
# Filter out deprecation warnings
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=SyntaxWarning, module="pysbd")

In [5]:
import os

SERPER_API_KEY = os.getenv('SERPER_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if SERPER_API_KEY and OPENAI_API_KEY:
    os.environ['SERPER_API_KEY'] = SERPER_API_KEY
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print('✅ API keys set successfully!')
else:
    raise ValueError('Please enter both SERPER and OPENAI API keys')

✅ API keys set successfully!


## Custom Guardrail Function
Here’s where the magic happens!

We're defining a custom validation function called **validate_blog_content**. This function checks the output of a task — in this case, we’re enforcing that the result is less than **{pick-your-word-count}** words.

If it passes the check, the task is complete.
If it fails, CrewAI will retry it (up to the number of retries you specify).

In [7]:
def validate_blog_content(result: TaskOutput) -> Tuple[bool, Any]:
    """Validate blog content meets requirements."""
    try:
        # Check word count
        try:
            word_count = len(result.raw.split())
            print(f"Word count: {word_count}")
            if word_count > 50:
                return (False, "Blog content exceeds 50 words")
        except Exception as wc_error:
            print(f"Error during word count check: {wc_error}")
            return (False, f"Error during word count check: {wc_error}")


        # Additional validation logic here
        return (True, result.raw.strip())
    except Exception as e:
        print(f"Unexpected error during validation: {e}")
        return (False, f"Unexpected error during validation: {e}")

## Define your Crew (Agent + Task + Tools)
Now we create a simple agent called "**Blog Writer**" whose goal is to generate blog content.

In [8]:
# LLM to be used by the agent(s)
llm = LLM(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

blog_agent = Agent(
    role="Blog Writer",
    goal="Write blog post",
    backstory="An expert blog writer",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=True
)

blog_task = Task(
    description="Write a super DETAILED blog post about {prompt} for {year}",
    expected_output="""A properly structured blog post under 50 words.
    Blog format:
    # Title
    ## Subtitle
    Paragraphs...
    """,
    agent=blog_agent,
    markdown=True,
    guardrail=validate_blog_content,  # Add the guardrail function
    max_retries=4  # Set the maximum number of retries
)

crew = Crew(
    agents=[blog_agent],
    tasks=[blog_task],
    verbose=True
)

## Launch the Crew
Finally, we bundle the agent and task into a **Crew**, pass in some inputs (*prompt* and *year*), and run it using **.kickoff()**.

In [9]:
results = crew.kickoff(
    inputs={
        "prompt": "CrewAI",
        "year": date.today().year
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 53c93fc0-e371-4f37-b56f-c30c73a7923d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Writer                                                                                             │
│                                                                                                                 │
│  Task: Write a super DETAILED blog post about CrewAI for 2025                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Writer                                                                                             │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"CrewAI 2025 features and updates\"}"                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'CrewAI 2025 features and updates', 'type': 'search', 'num': 10, 'engine':          │
│  'google'}, 'organic': [{'title': 'CrewAI Review 2025: The Right Sales Tool for Your Business?', 'link':        │
│  'https://reply.io/blog/crew-ai-review/', 'snippet': 'Key features include:', 'position': 1}, {'title': 'How    │
│  CrewAI is evolving beyond orchestration to create the most ...', 'link':                                       │
│  'https://blog.crewai.com/how-crewai-is-evolving-beyond-orchestration-to-create-the-most-powerful-agentic-ai-p  │
│  latform/', 'snippet': 'CrewAI has added a number of features to not support agentic RAG, but to provide        │
│  developers with greater flexibility. ... CrewAI © 2025.', 'position': 2}, {'title': 'CrewAI Pricing,           │
│  Features, & Alternatives Explained (2025) - Lindy', 'link': 'https://www.lindy.ai/blog/crew-ai-pricing',       │
│  'snippet': 'As your needs grow, you can upgrade to a higher plan with more executions, deployed crews, and     │
│  onboarding support. Annual billing discounts.', 'position': 3}, {'title': 'CrewAI', 'link':                    │
│  'https://www.crewai.com/', 'snippet': 'Streamline workflows across industries with powerful AI agents. Build   │
│  and deploy automated workflows using any LLM and cloud platform. Request a Demo.', 'position': 4,              │
│  'sitelinks': [{'title': 'CrewAI', 'link': 'https://blog.crewai.com/'}, {'title': 'Multi Agent Systems and      │
│  how...', 'link': 'https://learn.crewai.com/'}, {'title': 'Open source', 'link':                                │
│  'https://www.crewai.com/open-source'}, {'title': 'Documentation', 'link': 'https://docs.crewai.com/'}]},       │
│  {'title': 'New Release - 0.126.0 - Announcements - CrewAI', 'link':                                            │
│  'https://community.crewai.com/t/new-release-0-126-0/6112', 'snippet': 'CrewAI v0.126.0 is out!                 │
│  :police_car_light: Core Improvements & Fixes • Added support for Python 3.13 • Fixed agent knowledge sources   │
│  issue.', 'position': 5}, {'title': 'CrewAI Review 2025: Is It Really Worth Your Money? - Lindy', 'link':       │
│  'https://www.lindy.ai/blog/crew-ai', 'snippet': 'CrewAI is an open-source tool that lets developers make AI    │
│  agents. Y...                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🛡️ Guardrail Check ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Evaluation Started                                                                                   │
│  Name: def validate_blog_content(result: TaskOutput) -> T...                                                    │
│  Status: 🔄 Evaluating                                                                                          │
│  Attempt: 1                                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Word count: 50


╭────────────────────────────────────────────── 🛡️ Guardrail Success ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Passed                                                                                               │
│  Name: Validation Successful                                                                                    │
│  Status: ✅ Validated                                                                                           │
│  Attempts: 1                                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3cd83abd-4d22-41f2-a4e8-8f732f2fd073                                                                     │
│  Agent: Blog Writer                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 53c93fc0-e371-4f37-b56f-c30c73a7923d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # CrewAI in 2025                                                                                 │
│  ## The Future of Automated Workflows                                                                           │
│                                                                                                                 │
│  As **AI technologies continue to evolve**, CrewAI is at the forefront, enabling developers to create           │
│  intricate automated workflows.                                                                                 │
│                                                                                                                 │
│  Key features include:                                                                                          │
│  - Integration with leading LLMs                                                                                │
│  - Enhanced multi-agent collaboration                                                                           │
│  - Comprehensive support for Python                                                                             │
│                                                                                                                 │
│  Stay tuned for more updates!                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Displaying the Result
Once the Crew finishes executing, we display the final blog post using Markdown formatting:

In [10]:
from IPython.display import display, Markdown
display(Markdown(results.raw))

# CrewAI in 2025
## The Future of Automated Workflows

As **AI technologies continue to evolve**, CrewAI is at the forefront, enabling developers to create intricate automated workflows. 

Key features include:
- Integration with leading LLMs
- Enhanced multi-agent collaboration
- Comprehensive support for Python 

Stay tuned for more updates!

## Conclusion

And that’s it! You’ve now seen how to:

*   Write and register a task guardrail in CrewAI
*   Validate custom output logic
*   Automatically rerun tasks that don’t meet your standards

This makes your agents **more reliable**, **more controllable**, and **better suited to production use cases**.

👉 [Learn more in the docs](https://docs.crewai.com/en/concepts/tasks#task-guardrails)